## W&B Key - "4d4d4c8f520ba4f560db3da3a2e051523a812c46:"

In [1]:
!nvidia-smi

Sun Jan 11 01:32:51 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!git clone https://github.com/PerforatedAI/PerforatedAI.git
%cd PerforatedAI

# install
!pip install -e .

# Install dependencies
!pip install -q transformers datasets accelerate wandb

Cloning into 'PerforatedAI'...
remote: Enumerating objects: 1383, done.
remote: Counting objects: 100% (339/339), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 1383 (delta 249), reused 231 (delta 231), pack-reused 1044 (from 2)
Receiving objects: 100% (1383/1383), 72.27 MiB | 25.81 MiB/s, done.
Resolving deltas: 100% (764/764), done.
/content/PerforatedAI
Obtaining file:///content/PerforatedAI
  Preparing metadata (setup.py) ... done
  Running setup.py develop for perforatedai


## CIFAR-10 dataset start with new basline and dentrics

In [3]:
# =========================
# CELL 1 — Install + Imports
# =========================
!pip -q install -U sentence-transformers datasets wandb
!pip -q install -e .

import os, time
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

import wandb
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

from perforatedai import globals_perforatedai as GPA
from perforatedai import utils_perforatedai as UPA

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

WANDB_PROJECT = "perforatedai-sbert-stsb"

# Disable ipdb/pdb breaks
os.environ["PYTHONBREAKPOINT"] = "0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Building dendrites without Perforated Backpropagation
device: cuda


In [4]:
# =========================
# CELL 2 — Load STS-B
# =========================
ds = load_dataset("glue", "stsb")

def normalize_label(x):  # 0..5 -> 0..1
    return float(x) / 5.0

train_data = ds["train"]
val_data   = ds["validation"]

BATCH_SIZE = 32
MAX_LEN = 128

print("train:", len(train_data), "val:", len(val_data))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

stsb/train-00000-of-00001.parquet:   0%|          | 0.00/502k [00:00<?, ?B/s]

stsb/validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

stsb/test-00000-of-00001.parquet:   0%|          | 0.00/114k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

train: 5749 val: 1500


In [5]:
# ============================================
# CELL 3 — Build SBERT HF Backbone + Tokenizer
# ============================================
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

def build_sbert_encoder():
    sbert = SentenceTransformer(MODEL_NAME, device=device)
    hf_model = sbert._first_module().auto_model
    tokenizer = sbert.tokenizer
    return hf_model, tokenizer

base_model, tokenizer = build_sbert_encoder()
base_model = base_model.to(device)
print("HF model type:", type(base_model))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HF model type: <class 'transformers.models.bert.modeling_bert.BertModel'>


In [6]:
# =========================
# CELL 4 — Pooling + Dataloaders
# =========================
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state  # [B,T,H]
    mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    summed = torch.sum(token_embeddings * mask, dim=1)
    denom = torch.clamp(mask.sum(dim=1), min=1e-9)
    return summed / denom

def collate_fn(batch):
    s1 = [b["sentence1"] for b in batch]
    s2 = [b["sentence2"] for b in batch]
    y  = torch.tensor([normalize_label(b["label"]) for b in batch], dtype=torch.float32)

    t1 = tokenizer(s1, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
    t2 = tokenizer(s2, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")

    return {"t1": dict(t1), "t2": dict(t2), "labels": y}

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn)
val_loader   = DataLoader(val_data,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

In [7]:
# =========================
# CELL 5 — Loss + Spearman Eval
# =========================
def cosine_mse_loss(sim, y):
    return F.mse_loss(sim, y)

@torch.no_grad()
def evaluate_spearman(model):
    model.eval()
    preds, golds = [], []

    for batch in val_loader:
        t1 = {k: v.to(device) for k, v in batch["t1"].items()}
        t2 = {k: v.to(device) for k, v in batch["t2"].items()}
        y  = batch["labels"].cpu().numpy()

        out1 = model(**t1)
        out2 = model(**t2)

        emb1 = mean_pooling(out1, t1["attention_mask"])
        emb2 = mean_pooling(out2, t2["attention_mask"])
        emb1 = F.normalize(emb1, p=2, dim=1)
        emb2 = F.normalize(emb2, p=2, dim=1)

        sim = (emb1 * emb2).sum(dim=1).detach().cpu().numpy()

        preds.extend(sim.tolist())
        golds.extend(y.tolist())

    preds = np.array(preds)
    golds = np.array(golds)

    # simple Spearman (no scipy)
    def rankdata(a):
        temp = a.argsort()
        ranks = np.empty_like(temp)
        ranks[temp] = np.arange(len(a))
        return ranks

    rp = rankdata(preds)
    rg = rankdata(golds)
    cov = np.cov(rp, rg, bias=True)[0, 1]
    return float(cov / (rp.std() * rg.std() + 1e-9))

In [12]:
# ===================================================
# CELL 6 — PerforatedAI Fix: this_output_dimensions
# ===================================================
def fix_transformer_output_dims(model, device):
    """
    Transformer encoder Linear layers output [B, T, H].
    Tell PAI neuron dim = last dim (H) -> [-1, -1, 0].
    """
    vec = torch.tensor([-1, -1, 0], device=device)

    if hasattr(model, "encoder") and hasattr(model.encoder, "layer"):
        for layer in model.encoder.layer:
            # attention output dense
            if hasattr(layer, "attention"):
                if hasattr(layer.attention, "output") and hasattr(layer.attention.output, "dense"):
                    layer.attention.output.dense.set_this_output_dimensions(vec)

                # Q/K/V
                if hasattr(layer.attention, "self"):
                    if hasattr(layer.attention.self, "query"):
                        layer.attention.self.query.set_this_output_dimensions(vec)
                    if hasattr(layer.attention.self, "key"):
                        layer.attention.self.key.set_this_output_dimensions(vec)
                    if hasattr(layer.attention.self, "value"):
                        layer.attention.self.value.set_this_output_dimensions(vec)

            # FFN intermediate + output
            if hasattr(layer, "intermediate") and hasattr(layer.intermediate, "dense"):
                layer.intermediate.dense.set_this_output_dimensions(vec)

            if hasattr(layer, "output") and hasattr(layer.output, "dense"):
                layer.output.dense.set_this_output_dimensions(vec)

    return model

In [13]:
# ============================================
# CELL 7 — Baseline Training (no dendrites)
# ============================================
def train_baseline(model, lr=2e-5, epochs=3, run_name="baseline_no_dendrites"):
    wandb.init(project=WANDB_PROJECT, name=run_name, reinit="finish_previous")

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0

        for batch in train_loader:
            t1 = {k: v.to(device) for k, v in batch["t1"].items()}
            t2 = {k: v.to(device) for k, v in batch["t2"].items()}
            y  = batch["labels"].to(device)

            out1 = model(**t1)
            out2 = model(**t2)

            emb1 = mean_pooling(out1, t1["attention_mask"])
            emb2 = mean_pooling(out2, t2["attention_mask"])
            emb1 = F.normalize(emb1, p=2, dim=1)
            emb2 = F.normalize(emb2, p=2, dim=1)

            sim = (emb1 * emb2).sum(dim=1)
            loss = cosine_mse_loss(sim, y)

            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()

        spearman = evaluate_spearman(model)
        wandb.log({"epoch": epoch, "train/loss": total_loss / len(train_loader), "val/spearman": spearman})
        print(f"[Baseline] epoch={epoch} loss={total_loss/len(train_loader):.4f} spearman={spearman:.4f}")

    wandb.finish()
    return model

baseline_model, _ = build_sbert_encoder()
baseline_model = baseline_model.to(device)

baseline_model = train_baseline(baseline_model, lr=2e-5, epochs=1)
print("Baseline Spearman:", evaluate_spearman(baseline_model))

epoch,▁▅█
pai/params,▁▁▁
pai/restructured,▁▁▁
pai/training_complete,▁▁▁
train/loss,█▄▁
val/spearman,█▁▆
epoch,2
pai/params,33350784
pai/restructured,0
pai/training_complete,0
train/loss,0.01283


[Baseline] epoch=0 loss=0.0230 spearman=0.8938


epoch,▁
train/loss,▁
val/spearman,▁
epoch,0
train/loss,0.02296
val/spearman,0.89379


Baseline Spearman: 0.8937908967959494


In [14]:
# =========================================================
# CELL 8 — Dendritic Training (PerforatedAI, FIXED pooler)
# =========================================================
def train_dendritic(model, lr=2e-5, run_name="dendritic_with_pai", safety_max_epochs=200, warmup_epochs=1):
    wandb.init(project=WANDB_PROJECT, name=run_name, reinit="finish_previous")

    # PAI settings
    GPA.pc.set_unwrapped_modules_confirmed(True)
    GPA.pc.set_testing_dendrite_capacity(False)

    # IMPORTANT FIX: pooler is not used in our mean-pooling pipeline
    # so do NOT convert it to dendrites
    GPA.pc.append_module_ids_to_track([".pooler.dense"])

    # Recommended: no weight decay for dendrites
    WEIGHT_DECAY = 0.0

    # Convert (must be before .to(device))
    model = UPA.initialize_pai(model)

    # Fix transformer output dims so PAI doesn't abort
    model = fix_transformer_output_dims(model, device)

    model = model.to(device)

    # PAI-managed optimizer/scheduler
    GPA.pai_tracker.set_optimizer(torch.optim.AdamW)
    GPA.pai_tracker.set_scheduler(torch.optim.lr_scheduler.ReduceLROnPlateau)

    optim_args = {"params": model.parameters(), "lr": lr, "weight_decay": WEIGHT_DECAY}
    sched_args = {"mode": "max", "patience": 3, "factor": 0.5}

    optimizer, scheduler = GPA.pai_tracker.setup_optimizer(model, optim_args, sched_args)

    epoch = 0
    while True:
        model.train()
        total_loss = 0.0

        for batch in train_loader:
            t1 = {k: v.to(device) for k, v in batch["t1"].items()}
            t2 = {k: v.to(device) for k, v in batch["t2"].items()}
            y  = batch["labels"].to(device)

            out1 = model(**t1)
            out2 = model(**t2)

            emb1 = mean_pooling(out1, t1["attention_mask"])
            emb2 = mean_pooling(out2, t2["attention_mask"])
            emb1 = F.normalize(emb1, p=2, dim=1)
            emb2 = F.normalize(emb2, p=2, dim=1)

            sim = (emb1 * emb2).sum(dim=1)
            loss = cosine_mse_loss(sim, y)

            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()

        spearman = evaluate_spearman(model)

        # Warmup: don't let PAI switch too early
        if epoch >= warmup_epochs:
            model, restructured, training_complete = GPA.pai_tracker.add_validation_score(spearman, model)
            model = model.to(device)

            if restructured:
                optimizer, scheduler = GPA.pai_tracker.setup_optimizer(model, optim_args, sched_args)
        else:
            restructured = False
            training_complete = False

        wandb.log({
            "epoch": epoch,
            "train/loss": total_loss / len(train_loader),
            "val/spearman": spearman,
            "pai/restructured": int(restructured),
            "pai/training_complete": int(training_complete),
            "pai/params": sum(p.numel() for p in model.parameters()),
        })

        print(f"[Dendritic] epoch={epoch} loss={total_loss/len(train_loader):.4f} spearman={spearman:.4f} "
              f"restructured={restructured} complete={training_complete}")

        if training_complete:
            print("✅ PAI ended training (training_complete=True).")
            break

        epoch += 1
        if epoch >= safety_max_epochs:
            print("⚠️ Safety stop hit.")
            break

    wandb.finish()
    return model

dend_model, _ = build_sbert_encoder()
dend_model = dend_model.to(device)

dend_model = train_dendritic(dend_model, lr=2e-5, safety_max_epochs=200, warmup_epochs=1)
print("Dendritic Spearman:", evaluate_spearman(dend_model))

New list value of "_module_ids_to_track": ['.pooler.dense', '.pooler.dense']
Running Dendrite Experiment
[Dendritic] epoch=0 loss=0.0227 spearman=0.8946 restructured=False complete=False
Adding validation score 0.89420603
Checking PAI switch with mode n, switch mode DOING_HISTORY, epoch 0, last improved epoch 0, total epochs 0, n: 10, num_cycles: 0
Returning False - no triggers to switch have been hit
[Dendritic] epoch=1 loss=0.0166 spearman=0.8942 restructured=0 complete=False
Adding validation score 0.89366498
Checking PAI switch with mode n, switch mode DOING_HISTORY, epoch 1, last improved epoch 1, total epochs 1, n: 10, num_cycles: 0
Returning False - no triggers to switch have been hit
[Dendritic] epoch=2 loss=0.0129 spearman=0.8937 restructured=0 complete=False
Adding validation score 0.89292054
Checking PAI switch with mode n, switch mode DOING_HISTORY, epoch 2, last improved epoch 1, total epochs 2, n: 10, num_cycles: 0
Returning False - no triggers to switch have been hit
[De

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
pai/params,▁▁▁▁▁▁▁▁▁▁▁▁███████████████████████▁
pai/restructured,▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁█
pai/training_complete,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
train/loss,█▆▅▄▃▂▂▂▂▁▁▁▁▄▃▂▂▂▂▁▁▁▁▁▁▄▃▂▂▂▁▁▁▁▁▁
val/spearman,█▇▇▅▆▄▃▃▃▃▃▂▃▆▆▅▄▅▃▆▅▃▃▃▅▆▂▂▂▃▃▃▂▁▂▃
epoch,35
pai/params,33350784
pai/restructured,1
pai/training_complete,1
train/loss,0.00331


Dendritic Spearman: 0.8936649802955421
